In [6]:
def is_variable(item):
    return item.startswith('?') and all(s.isalpha() for s in item[1:])

def pattern_match(pattern, saying):
    if not pattern or not saying: 
        return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pattern_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pattern_match(pattern[1:], saying[1:])

In [7]:
pattern_match('I want ?X'.split(), "I want holiday".split())

[('?X', 'holiday')]

In [8]:
patterns = pattern_match("?X greater than ?Y".split(), "3 greater than 2".split())
print(patterns)

[('?X', '3'), ('?Y', '2')]


In [9]:
def pattern_to_dic(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}
print(pattern_to_dic(patterns))

{'?X': '3', '?Y': '2'}


In [57]:
#['?x你好呀?y'] {'?x': 'Alexa', '?y': 'Maya'}
def subsidy(rule, parsed_pattern):
    if not rule: return []
    return [parsed_pattern.get(rule[0], rule[0])] + subsidy(rule[1:], parsed_pattern)

In [58]:
got_patterns = pattern_match("I want ?X".split(), "I want iPhone".split())
#print(got_patterns)
answer = subsidy('What does it mean if you got a ?X'.split(), pattern_to_dic(got_patterns))
print(answer)
join_answer = ' '.join(answer)
print(join_answer)

['What', 'does', 'it', 'mean', 'if', 'you', 'got', 'a', 'iPhone']
What does it mean if you got a iPhone


In [12]:
got_pattern = pattern_match('?P needs ?X'.split(), 'John needs resting'.split())
print(got_pattern)
answer = subsidy('Why does ?P need ?X'.split(), pattern_to_dic(got_pattern))
join_answer = ' '.join(answer)
print(join_answer)

[('?P', 'John'), ('?X', 'resting')]
Why does John need resting


In [13]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}


In [14]:
import random
choice = random.choice

def get_response(saying, rules):
    if (not saying) or (not rules): return []
   
    # find the matching pattern
    for pattern, answer_pattern in defined_patterns.items():
        matched_pattern = pattern_match(pattern.split(), saying.split())
        if not matched_pattern: continue
        else:
            return ' '.join(subsidy((choice(answer_pattern)).split(), pattern_to_dic(matched_pattern)))
            
print(get_response('I need iPhone', defined_patterns))
print(get_response("My mother told me something", defined_patterns))

Image you will get iPhone soon
Talk about more about your mother


In [126]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

def is_match(rest, saying):
    #print(rest, saying)
    if not rest or not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return ('', ''), 0

In [127]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())


(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [128]:
fail = [True, None]
def pat_match_with_seg(pattern, saying):
    #print('pattern: {}, saying: {}'.format(pattern, saying))
    if not pattern or not saying: return []
    pat = pattern[0]
    
    if is_variable(pat):
        return [pat, saying[0]] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        #print(match)
        if index == 0: #no match
            return fail
        return [match]+ pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [129]:
print(pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split()))
pat_match_with_seg('?*X hello ?*Y'.split(), 'I was smart'.split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]


[True, None]

In [130]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

pat_match_with_seg('I need ?*X'.split(), 'I need an iPhone'.split())

[('?X', ['an', 'iPhone'])]

In [131]:
words = subsidy("Why do you need ?X".split(), pattern_to_dic(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))
' '.join(words)

words = subsidy("I like ?X".split(), pattern_to_dic(pat_match_with_seg('?*X is cute'.split(), 
                  "my cat is cute".split())))
' '.join(words)


'I like my cat'

In [132]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

def get_response(saying, response_rules):
    if not saying or not response_rules: return ''
    
    for pattern, response in response_rules.items():
        match = pat_match_with_seg(pattern.split(), saying.split())
        if fail == match:
            continue
        else:
            #print(match)
            return ' '.join(subsidy(choice(response).split(), pattern_to_dic(match)))
    return ''

In [133]:
get_response('I was good at table tennis', rules)

'Were you really good at table tennis ?'

In [134]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x not ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']
}

print(get_response('Alexa, hello', rule_responses));
print(get_response('Alexa, I want a holiday', rule_responses));

Please state your problem
what would it mean if you got a holiday


In [157]:
import re
def is_chinese_sentence(sentence):
    chinese_model = re.compile(u'[\u4e00-\u9fa5]')
    return None != chinese_model.search(sentence)

import jieba
jieba.load_userdict('./dict/userdict.txt')

def cut(string):
    return list(jieba.cut(string, cut_all = False))
 
# parse both English and Chinese 
def get_response_ext(saying, response_rules):
    if not saying or not response_rules: return ''
    
    is_saying_chinese = False
    if is_chinese_sentence(saying):
        is_saying_chinese = True
    
    for pattern, response in response_rules.items():
        # check if language matches
        if is_chinese_sentence(pattern) != is_saying_chinese:  #language doesn't match, skip
            continue
        
        if is_saying_chinese:
            match =  pat_match_with_seg(cut(pattern), cut(saying))
        else:
            match = match = pat_match_with_seg(pattern.split(), saying.split())

        #print(match)
        if fail == match:
            continue
        elif is_saying_chinese:
            return ''.join( subsidy( cut(choice(response)), pattern_to_dic(match) ))
        else:
            return ' '.join(subsidy(choice(response).split(), pattern_to_dic(match)))
    return ''

In [163]:
chinese_rule_responses = {
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x,你好我是?*y': ['你好呀?y', '?y请告诉我你的问题'],
    '?*x,我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了']
}

rule_responses_mix = {
    '?*x hello I am ?*y': ['How do you do, ?y', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

def generate_responses(sayings):
    for saying in sayings:
        print(get_response_ext(saying, rule_responses_mix))

sayings = [
    'Alexa, I was good at table tennis',
    'Alexa hello I am Maya',
    'Alexa,你好我是Maya',
    '小爱,我想滑雪'
]

generate_responses(sayings)


Perhaps I already knew you were good at table tennis
How do you do, Maya
请告诉我你的问题
你可以想想你很快就可以滑雪了
